In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from tabulate import tabulate
from tqdm import tqdm
from pprint import pformat
import pickle

/home/adeel/anaconda3/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
df = pd.read_csv('./Inventario Rutas PDV 2018-2032.csv')
df

,ID,Tramo_COD,Ruta,Descripción_Tramo,Cat_tramo,Longitud,Cat_Ruta,Inicio_Tramo,Final_Tramo,longitud_shape,...,Unnamed: 251,Unnamed: 252,Nombre_Eje_Concesión,% APP,Nombre_eje_logístico,Tipo_Sección_Final,Actuación final,Coste actuación final DGC,Coste actuación final APP,APP Nova via
0,1,CA-01-ORIENTE-05,CA-1-E,"CA-01-ORIENTE-05 -FCA LA CONCHA - RD-SRO-04, D...",RED PRIMARIA,5.59999990463257,RUTAS CENTROAMERICANAS,6+000,165+200,0.048469396870762,...,NaN,NaN,CA1 Oriente,1,CPL3,Tipo A+,Ampliar capacidad,0,163.575997214317,0
1,2,CA-01-ORIENTE-12,CA-1-E,CA-01-ORIENTE-12 -DESVIO RD-JUT-03-A - JUTIAPA,RED PRIMARIA,16.6800003051758,RUTAS CENTROAMERICANAS,6+000,165+200,0.160232205501041,...,NaN,NaN,Anillo Regional Tramo 7,1,CPL3,Tipo A,Ampliar capacidad,0,274.319151498614,0
2,3,CA-02-OCCIDENTE-09A,CA-02-W6,"DESVIO SAN ANTONIO, BIFURCACION CA-02-OCCIDENT...",RED SECUNDARIA,16.3999996185303,RUTAS CENTROAMERICANAS,147+300,149+000,0.169761386611261,...,NaN,NaN,CA2 Occidente,1,CPL2,Tipo A,Ampliar capacidad,0,208.279995155334,0
3,4,CA-02-OCCIDENTE-09B,CA-02-W4,"KM. 165+075, HACIA CUYOTENANGO - BIFURCACION R...",RED PRIMARIA,3.9300000667572,RUTAS CENTROAMERICANAS,163+300,281+210,0.034619635415597,...,NaN,NaN,CA2 Occidente,1,CPL2,Tipo A+,Ampliar capacidad,0,99.8220016956329,0
4,5,CA-02-OCCIDENTE-10,CA-02-W4,CA-02-OCCIDENTE-10 -DESVIO SAN ANDRES VILLA SE...,RED PRIMARIA,10.5699996948242,RUTAS CENTROAMERICANAS,163+300,281+210,0.090497879147784,...,NaN,NaN,CA2 Occidente,1,CPL2,Tipo A,Ampliar capacidad,0,154.374845542908,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,1096,NC_08,0,Nueva RN-07-W para conectar la CA-01-w con RN-...,RED TERCIARIA,40.5921211242676,RUTAS DEPARTAMENTALES,0,0,0.371357509988736,...,NaN,NaN,0,0,0,Tipo D,Nueva,211.079029846191,0,0
1096,1097,NC_15,0,Nueva RD para acceder a Sipacapa,RED TERCIARIA,10.1757345199585,RUTAS DEPARTAMENTALES,0,0,0.093992235805604,...,NaN,NaN,0,0,0,Tipo D,Nueva,52.9138195037842,0,0
1097,1098,NC_06,SM-1,Conexión Tejutla-Comitancillo-San Lorenzo,RED TERCIARIA,24.9044628143311,RUTAS DEPARTAMENTALES,0,0,0.228171677257774,...,NaN,NaN,0,0,0,Tipo D,Nueva,129.503206634521,0,0
1098,1099,NC_07,0,Prolongación RD-SM para conectar San Lorenzo c...,RED TERCIARIA,5.70658731460571,RUTAS DEPARTAMENTALES,0,0,0.053046271862806,...,NaN,NaN,0,0,0,Tipo D,Nueva,29.6742540359497,0,0


## Translate column names

In [3]:
from argostranslate import package, translate
package.install_from_path('es_en.argosmodel')
installed_languages = translate.get_installed_languages()
[str(lang) for lang in installed_languages]
translator = installed_languages[1].get_translation(installed_languages[0])


In [6]:
translation_overrides = {
    'iri': 'iri'
}

In [8]:
columns = [c.replace('_', ' ').strip().lower() for c in df.columns.to_list()]
col_es_to_en = {c: translator.translate(c) for c in columns}
col_es_to_en.update(translation_overrides)
df.columns = list(col_es_to_en.values())

In [10]:
print(tabulate(zip(columns[:40], df.columns[:40])))

-------------------------------  ---------------------------
id                               id
tramo cod                        stretch
ruta                             route
descripción tramo                description of the section
cat tramo                        cat stretch
longitud                         length
cat ruta                         cat route
inicio tramo                     start stage
final tramo                      final stage
longitud shape                   length shape
pki                              pki
pkf                              pkf
departamento                     Department
municipio                        municipality
entorno urbano                   urban environment
nueva vía                        new
0 si es vía existente            0 if it is an existing path
1 si es nueva vía                1 if it is new
núm carriles                     No lanes
ancho calzada                    width
ancho hombros                    wide shoulders
ancho pla

In [11]:
df.to_csv('wb_data_en.csv')

## Data cleaning

In [12]:
import unicodedata as ud

# https://stackoverflow.com/a/15547803/5908685
def rmdiacritics(char):
    '''
    Return the base character of char, by "removing" any
    diacritics like accents or curls and strokes and the like.
    '''
    desc = ud.name(char)
    cutoff = desc.find(' WITH ')
    if cutoff != -1:
        desc = desc[:cutoff]
        try:
            char = ud.lookup(desc)
        except KeyError:
            pass  # removing "WITH ..." produced an invalid name
    return char

def remove_diacritics(s: str):
    return ''.join(map(rmdiacritics, s))

# 'Palá --> ' + remove_diacritics('Palá')

In [13]:
df = pd.read_csv('wb_data_en.csv')
df.loc[:, 'description'] = df['description of the section']
df.loc[:, 'description'] = df.description.map(str.lower).map(remove_diacritics)
df.loc[:, 'description'] = df.description.str.replace('bif.', 'bifurcacion', regex=False)
df.loc[:, 'description'] = df.description.str.replace('bif ', 'bifurcacion ', regex=False)
df.loc[:, 'description'] = df.description.str.replace('fca.', 'finca', regex=False)
df.loc[:, 'description'] = df.description.str.replace('fca ', 'finca ', regex=False)
df.loc[:, 'description'] = df.description.str.replace(r' -(\w)', r' \1', regex=True)


## Simple cases

\<place\> - \<palce\>

In [14]:
rgx = r'^[\s\w]+? - [\s\w]+?$'
simple_mask = df.description.str.contains(rgx)
df_simple = pd.DataFrame(df[simple_mask])
df_simple

,Unnamed: 0,id,stretch,route,description of the section,cat stretch,length,cat route,start stage,final stage,...,unnamed: 252,name axis concession,% app,logistic axis name,type final section,Final performance,cost performance final dgc,cost performance final app,app nova via,description
15,15,16,CA-09-NORTE-01,CA-09-N,PUENTE BELICE - PUENTE RODRIGUITOS,RED PRIMARIA,4.46999979019165,RUTAS CENTROAMERICANAS,3+920,292+200,...,NaN,CA9 Norte,1,CPL1,Tipo A+,Ampliar capacidad,0,67.0499968528748,0,puente belice - puente rodriguitos
18,18,19,CA-09-NORTE-20B,CA-09-N,LA RUIDOSA - BIFURCACION CAYUGA,RED PRIMARIA,14.8900003433228,RUTAS CENTROAMERICANAS,3+920,292+200,...,NaN,CA9 Norte,1,CPL1,Tipo A,Ampliar capacidad,0,189.103004360199,0,la ruidosa - bifurcacion cayuga
19,19,20,CA-09-NORTE-21,CA-09-N,BIFURCACION CAYUGA - BIFURCACION EL CIMARRON,RED PRIMARIA,9.80000019073486,RUTAS CENTROAMERICANAS,3+920,292+200,...,NaN,CA9 Norte,1,CPL1,Tipo A,Ampliar capacidad,0,158.749977697796,0,bifurcacion cayuga - bifurcacion el cimarron
28,28,29,CA-09-SUR-05,CA-09-S1,500 METROS ADELANTE BASCULA DGC - PALIN,RED PRIMARIA,8.94999980926514,RUTAS CENTROAMERICANAS,8+500,38+200,...,NaN,CA9 Sur,1,CPL1,Tipo A+,Ampliar capacidad,0,134.249997138977,0,500 metros adelante bascula dgc - palin
29,29,30,CA-09-SUR-11,CA-09-S3,PUERTO QUETZAL - PUERTO IZTAPA,RED PRIMARIA,9.75,RUTAS CENTROAMERICANAS,111+000,124+000,...,NaN,0,0,0,Tipo B,Mantenimiento menor,0.39,0,0,puerto quetzal - puerto iztapa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,1066,1067,NC_Radial_5,0,Nueva Radial entre anillos - Norte,RED PRIMARIA,8.74459075927734,RUTAS NACIONALES,0,0,...,NaN,0,0,0,Tipo B,Nueva,104.0606300354,0,0,nueva radial entre anillos - norte
1067,1067,1068,NC_Radial_1,0,Nueva Radial entre anillos - Este,RED PRIMARIA,19.9354286193848,RUTAS NACIONALES,0,0,...,NaN,0,0,0,Tipo B,Nueva,237.231600570679,0,0,nueva radial entre anillos - este
1068,1068,1069,NC_Radial_2,0,Nueva Radial entre anillos - Sud Este,RED PRIMARIA,26.5027656555176,RUTAS NACIONALES,0,0,...,NaN,0,0,0,Tipo B,Nueva,315.382911300659,0,0,nueva radial entre anillos - sud este
1069,1069,1070,NC_Radial_3,0,Nueva Radial entre anillos - Sud,RED PRIMARIA,22.825891494751,RUTAS NACIONALES,0,0,...,NaN,0,0,0,Tipo B,Nueva,271.628108787537,0,0,nueva radial entre anillos - sud


In [15]:
np.mean(simple_mask)

0.29

### Parse endpoints

In [16]:
endpoint_1 = [p for (_, (p, _)) in df_simple.description.str.split(' - ').items()]
endpoint_2 = [p for (_, (_, p)) in df_simple.description.str.split(' - ').items()]
df_simple.loc[:, 'endpoint_1'] = endpoint_1
df_simple.loc[:, 'endpoint_2'] = endpoint_2

In [17]:
df_simple.loc[:, ['description', 'endpoint_1', 'endpoint_2']]

,description,endpoint_1,endpoint_2
15,puente belice - puente rodriguitos,puente belice,puente rodriguitos
18,la ruidosa - bifurcacion cayuga,la ruidosa,bifurcacion cayuga
19,bifurcacion cayuga - bifurcacion el cimarron,bifurcacion cayuga,bifurcacion el cimarron
28,500 metros adelante bascula dgc - palin,500 metros adelante bascula dgc,palin
29,puerto quetzal - puerto iztapa,puerto quetzal,puerto iztapa
...,...,...,...
1066,nueva radial entre anillos - norte,nueva radial entre anillos,norte
1067,nueva radial entre anillos - este,nueva radial entre anillos,este
1068,nueva radial entre anillos - sud este,nueva radial entre anillos,sud este
1069,nueva radial entre anillos - sud,nueva radial entre anillos,sud


In [18]:
df_simple.to_csv('simple_cases.csv')